<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/PAIassignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
import shutil

from google.colab import drive

In [2]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the dataset folder
dataset_path = '/content/drive/MyDrive/archive1'  # Replace with your actual folder path

# Verify the folder structure by listing files
import os
print("Dataset folders:", os.listdir(dataset_path))  # Should show 'train' and 'test' folders

# Paths to train and test folders
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')

# Verify train and test folders
print("Train folder contents:", os.listdir(train_path))
print("Test folder contents:", os.listdir(test_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset folders: ['train', 'test']
Train folder contents: ['NonDemented', 'Demented']
Test folder contents: ['Demented', 'NonDemented']


In [3]:
# Step 2: Define image dimensions and batch size
image_size = (128, 128)
batch_size = 32

# Data augmentation setup for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,           # Normalize pixel values to [0, 1]
    rotation_range=30,           # Random rotation up to 30 degrees
    width_shift_range=0.2,       # Horizontal shift
    height_shift_range=0.2,      # Vertical shift
    shear_range=0.2,             # Shear transformation
    zoom_range=0.2,              # Random zoom
    horizontal_flip=True,        # Flip images horizontally
    fill_mode='nearest'          # Fill missing pixels after transformation
)

# Data generator for the test set (without augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_path,                   # Use the train folder path
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',          # For binary classification
    color_mode='rgb',
    shuffle=True
)

# Load and preprocess testing data
test_generator = test_datagen.flow_from_directory(
    test_path,                    # Use the test folder path
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)


Found 4861 images belonging to 2 classes.
Found 1603 images belonging to 2 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
# Import the Rescaling layer correctly
from tensorflow.keras.layers import Rescaling  # Or tf.keras.layers.experimental.preprocessing.Rescaling for older versions

# ... rest of your code ...

In [5]:
# Step 2: Define the CNN Model Architecture
model = Sequential([
    # Input Layer with Rescaling
    Rescaling(1.0 / 255, input_shape=(128, 128, 3)),

    # First Convolutional Block
    Conv2D(16, (3, 3), activation='relu'),                  # Conv2D with 16 filters
    MaxPooling2D(pool_size=(2, 2)),                         # MaxPooling with pool size 2x2

    # Second Convolutional Block
    Conv2D(32, (3, 3), activation='relu'),                  # Conv2D with 32 filters
    MaxPooling2D(pool_size=(2, 2)),                         # MaxPooling with pool size 2x2
    Dropout(0.2),                                           # Dropout with 0.2 rate

    # Third Convolutional Block
    Conv2D(64, (3, 3), activation='relu'),                  # Conv2D with 64 filters
    MaxPooling2D(pool_size=(2, 2)),                         # MaxPooling with pool size 2x2
    Dropout(0.25),                                          # Dropout with 0.25 rate

    # Flatten Layer
    Flatten(),                                              # Flatten to convert feature maps to a vector

    # Fully Connected Dense Layers
    Dense(128, activation='relu'),                          # Dense layer with 128 units
    Dense(64, activation='relu'),                           # Dense layer with 64 units

    # Output Layer for Binary Classification
    Dense(1, activation='sigmoid')                          # Output layer with 1 unit and sigmoid activation
])

# Step 3: Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 4: Train the Model
history = model.fit(
    train_generator,
    epochs=100,  # Changed to 100 epochs
    validation_data=test_generator,
    verbose=1
)

# Step 5: Save the Trained Model
model.save('/content/drive/MyDrive/alzheimers_cnn_model.h5')

# Print Model Summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


152/152 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.4874 - loss: 0.6932 - val_accuracy: 0.5022 - val_loss: 0.6931
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 121s 776ms/step - accuracy: 0.5018 - loss: 0.6932 - val_accuracy: 0.5022 - val_loss: 0.6931
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 140s 771ms/step - accuracy: 0.4897 - loss: 0.6932 - val_accuracy: 0.4978 - val_loss: 0.6931
Epoch 4/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 120s 766ms/step - accuracy: 0.4802 - loss: 0.6932 - val_accuracy: 0.4978 - val_loss: 0.6931
Epoch 5/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 130s 843ms/step - accuracy: 0.5029 - loss: 0.6932 - val_accuracy: 0.4978 - val_loss: 0.6932
Epoch 6/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 135s 790ms/step - accuracy: 0.5038 - loss: 0.6932 - val_accuracy: 0.4978 - val_loss: 0.6931
Epoch 7/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 140s 775ms/step - accuracy: 0.4893 - loss: 0.6932 - val_accuracy: 0.5022 - val_loss: 0.6931
Epoch 8/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 123s 792ms/step - accuracy: 0.5232 - loss: 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 126, 126, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,912,997 (18.74 MB)

 Trainable params: 1,637,665 (6.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,275,332 (12.49 MB)